In [ ]:
import comet_ml
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from torch.utils.data import DataLoader, random_split

from datasets import Poly, get_wine_data
from models import ModelSmall, ModelMicro
from ensemble import predict, evaluate

load_dotenv()
comet_ml.login()
api = comet_ml.API()

# Ensemble Member Summary

In [ ]:
def get_ensemble_df(name: str) -> pd.DataFrame:
    data = {"key": [], "train_loss": [], "val_loss": []}

    for experiment in api.get("nichlas-jacobs/pdl-hw2"):
        if experiment.name == name:
            data["key"].append(experiment.key)
            data["train_loss"].append(
                float(experiment.get_metrics("train_loss")[-1]["metricValue"])
            )
            data["val_loss"].append(
                float(experiment.get_metrics("val_loss")[-1]["metricValue"])
            )

    return pd.DataFrame(data)


df = get_ensemble_df("Wine_Trad_Ensemble_600_32")
keys = df["key"].to_list()

print("All Ensembles")
print(df)

print("\ntrain_loss description:")
print(df["train_loss"].describe())

print("\nval_loss description:")
print(df["val_loss"].describe())

# Load Dataset

In [ ]:
train_ds, test_ds = get_wine_data()
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

# Load the Models

In [ ]:
device = torch.device("cuda:1")
models = []

for key in keys:
    model = ModelSmall(input_dim=11, hidden_dim=32)
    model.load_state_dict(torch.load(f"../checkpoints/{key}.pt"))
    models.append(model.to(device))

    print(
        evaluate(train_ds, device, [model.to(device)])[0],
        evaluate(test_ds, device, [model.to(device)])[0],
    )

# Ensemble Performance

In [ ]:
print("Train performance: MSE_mean, MSE_median, average SD")
print(evaluate(train_ds, device, models))

print("Test performance: MSE_mean, MSE_median, average SD")
print(evaluate(test_ds, device, models))